In [1]:
import mido
import numpy as np
import matplotlib.pyplot as plt
import PreProcessor as pp
from keras.models import Model
from tensorflow.keras.layers import MultiHeadAttention, Embedding, Dense, LayerNormalization, Dropout, Softmax, concatenate, Add
from keras import optimizers
import tensorflow as tf
from utils import *

In [2]:
dataset = pp.load_dataset("../adl-piano-midi")
ClassicSongs = pp.files_to_songs(dataset["Classical"])

channel_range, note_range, velocity_range, time_range = pp.ranges_from_songs(ClassicSongs,True,True,True,True)

n_Channels = int(channel_range[1])+1
n_Notes = int(note_range[1])+1
n_Velocities = int(velocity_range[1])+1

print("\nNumber of channels   :",n_Channels,"\nNumber of notes      :",n_Notes,"\nNumber of velocities :",n_Velocities,"\nTime range           :",time_range[0],time_range[1])


Number of channels   : 13 
Number of notes      : 120 
Number of velocities : 128 
Time range           : 0.0 190.285


In [3]:
seq_length = 30
val_split = 0.1
test_split = 0.1

Train_X, Train_y, Val_X, Val_y, Test_X, Test_y = pp.prep_data_transformer(seq_length,ClassicSongs,n_Channels,n_Notes,n_Velocities,val_split,test_split)

In [4]:
class MusicEmbedding(tf.keras.layers.Layer):
    def __init__(self, n_Channels, n_Notes, n_Velocities, d_model, dff, dropout_rate=0.1):
        super().__init__()
        self.d_model = d_model
        
        tot_dim = n_Channels + n_Notes + int(n_Velocities/4)
        self.d_Channels = int((d_model-1)*n_Channels/tot_dim)
        self.d_Notes = int((d_model-1)*n_Notes/tot_dim)
        self.d_Velocities = int((d_model-1)*int(n_Velocities/4)/tot_dim)
        while self.d_Channels + self.d_Notes + self.d_Velocities != d_model - 1 : self.d_Channels += 1
            
        self.Channel_Embedding = Embedding(n_Channels, self.d_Channels)
        self.Notes_Embedding = Embedding(n_Notes, self.d_Notes)
        self.Velocities_Embedding = Embedding(n_Velocities, self.d_Velocities)
        self.seq = tf.keras.Sequential([
          Dense(dff, activation='relu'),
          Dense(d_model),
          Dropout(dropout_rate)
        ])

    def call(self, x):
        chan = self.Channel_Embedding(x[0])
        note = self.Notes_Embedding(x[1])
        velo = self.Velocities_Embedding(x[2])
        
        # Scale to values between 0 and 1 ? (/time_range[1])
        time = tf.expand_dims(x[3],-1)
        out = concatenate([chan,note,velo,time])
        return self.seq(out)

In [5]:
class PositionalEncoding(tf.keras.layers.Layer):
    def __init__(self, d_model, seq_length):
        super().__init__()
        self.d_model = d_model
        self.seq_length = seq_length
        self.pos_encoding = self.positional_encoding(seq_length, d_model)
        
    def positional_encoding(self, length, depth):
        depth = depth/2

        positions = np.arange(length)[:, np.newaxis]     # (seq, 1)
        depths = np.arange(depth)[np.newaxis, :]/depth   # (1, depth)

        angle_rates = 1 / (10000**depths)         # (1, depth)
        angle_rads = positions * angle_rates      # (pos, depth)

        pos_encoding = np.concatenate([np.sin(angle_rads), np.cos(angle_rads)],axis=-1) 

        return tf.cast(pos_encoding, dtype=tf.float32)[tf.newaxis, :, :]

    def call(self, x):
        # This factor sets the relative scale of the embedding and positonal_encoding.
        # x *= tf.math.sqrt(tf.cast(self.d_model, tf.float64))
        return x + self.pos_encoding

In [6]:
class SelfAttentionBlock(tf.keras.layers.Layer):
    # Dropout ??
    def __init__(self, num_heads, d_model, **kwargs):
        super().__init__()
        self.mha = MultiHeadAttention(num_heads, d_model, **kwargs)
        self.layer_norm = LayerNormalization()
        self.add = Add()

    def call(self, x):
        attn_output = self.mha(query=x,value=x,key=x,use_causal_mask = True)
        x = self.add([x, attn_output])
        x = self.layer_norm(x)
        return x

In [7]:
class FeedForward(tf.keras.layers.Layer):
    def __init__(self, d_model, dff, dropout_rate=0.1):
        super().__init__()
        self.seq = tf.keras.Sequential([
          Dense(dff, activation='relu'),
          Dense(d_model),
          Dropout(dropout_rate)
        ])
        self.add = Add()
        self.layer_norm = LayerNormalization()

    def call(self, x):
        x = self.add([x, self.seq(x)])
        x = self.layer_norm(x) 
        return x

In [8]:
class DecoderLayer(tf.keras.layers.Layer):
    def __init__(self,d_model,num_heads,dff,dropout_rate=0.1):
        super().__init__()
        self.self_attention_block = SelfAttentionBlock(num_heads,d_model)
        self.ffn = FeedForward(d_model, dff)

    def call(self, x):
        x = self.self_attention_block(x)
        x = self.ffn(x)
        return x

In [12]:
class Transformer(tf.keras.Model):
    def __init__(self, n_Channels, n_Notes, n_Velocities, max_time, seq_length, d_model, n_layers, n_heads, dff, dropout_rate=0.1):
        super().__init__()
        self.embedding = MusicEmbedding(n_Channels, n_Notes, n_Velocities, d_model, dff, dropout_rate)
        self.pos_enc = PositionalEncoding(d_model, seq_length)
        self.layer_norm = LayerNormalization()
        self.dec_layers = [DecoderLayer(d_model, n_heads, dff, dropout_rate) for _ in range(n_layers)]
        self.out_chan = Dense(n_Channels, activation="softmax",name="Channels")
        self.out_note = Dense(n_Notes, activation="softmax",name="Notes")
        self.out_velo = Dense(1, activation = lambda x: (tf.sigmoid(x) * n_Notes),name="Velocities")
        self.out_time = Dense(1, activation = lambda x: ((1 + tf.tanh(x)) * 0.5 * max_time),name="Times")
    
    def call(self, inputs):
        x = self.embedding(inputs)
        x = self.pos_enc(x)
        x = self.layer_norm(x)
        for dec_layer in self.dec_layers:
            x = dec_layer(x)
        chan = self.out_chan(x)
        note = self.out_note(x)
        velo = self.out_velo(x)
        time = self.out_time(x)
        return [chan, note, velo, time]
    
    def generate(self, input_vec, output_length, mode="Max", T=1, reset=False):
        temp_vec = self.predict(input_vec)
        temp_vec = sample_probas(temp_vec,mode,T)
        output_vec = [[np.argmax(temp_vec[0][0,0,:])],[np.argmax(temp_vec[1][0,0,:])],[temp_vec[2][0,0,0]],[temp_vec[3][0,0,0]]]
        for _ in range(output_length-1):
            for i in range(4):
                input_vec[i] = np.concatenate((input_vec[i][:,1:],np.reshape([output_vec[i][-1]],(1,1))),axis=1)
            temp_vec = self.predict(input_vec)
            temp_vec = sample_probas(temp_vec,mode,T)
            output_vec[0] += [np.argmax(temp_vec[0][0,0,:])]
            output_vec[1] += [np.argmax(temp_vec[1][0,0,:])]
            output_vec[2] += [temp_vec[2][0,0,0]]
            output_vec[3] += [temp_vec[3][0,0,0]]
        return (output_vec[0],output_vec[1],output_vec[2],output_vec[3])

In [13]:
d_model = 128
n_layers = 2
n_heads = 6
dff = 4*d_model

n_batch = 200
learning_rate = 0.01

optimizer= optimizers.Adam(learning_rate=learning_rate)

losses = {
    'output_1': 'categorical_crossentropy',    # Channels
    'output_2': 'categorical_crossentropy',    # Notes
    'output_3': 'mean_squared_error',          # Velocities
    'output_4': 'mean_squared_error'           # Times
}


loss_weights_start = {
    'output_1': 1.0,      # Channels
    'output_2': 1.0,      # Notes
    'output_3': 1.0,      # Velocities
    'output_4': 1.0       # Times
}

loss_weights_then  = {
    'output_1': 0.5,     # Channels
    'output_2': 5.0,     # Notes
    'output_3': 1e-3,    # Velocities
    'output_4': 1e-6     # Times
}

metrics = {
    'output_1': 'accuracy',          # Channels
    'output_2': 'accuracy',          # Notes
    'output_3': rounded_accuracy,    # Velocities
    'output_4': tol_accuracy         # Times
}

ClassicTransformer = Transformer(n_Channels, n_Notes, n_Velocities, time_range[1], seq_length, d_model, n_layers, n_heads, dff)
build = ClassicTransformer([Train_X[0][0:2,:],Train_X[1][0:2,:],Train_X[2][0:2,:],Train_X[3][0:2,:]])
ClassicTransformer.summary()
ClassicTransformer.compile(optimizer=optimizer, loss=losses, loss_weights=loss_weights_start, metrics=metrics)

loss = []
channels_accuracy = []
notes_accuracy = []
velocities_accuracy = []
times_accuracy = []

val_loss = []
val_channels_accuracy = []
val_notes_accuracy = []
val_velocities_accuracy = []
val_times_accuracy = []

Model: "transformer_1"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 music_embedding_1 (MusicEmb  multiple                 145967    
 edding)                                                         
                                                                 
 positional_encoding_1 (Posi  multiple                 0         
 tionalEncoding)                                                 
                                                                 
 layer_normalization_5 (Laye  multiple                 256       
 rNormalization)                                                 
                                                                 
 decoder_layer_2 (DecoderLay  multiple                 527872    
 er)                                                             
                                                                 
 decoder_layer_3 (DecoderLay  multiple               

In [14]:
Chan, Note, Velo, Time = ClassicTransformer.generate([Train_X[0][0:1,:],Train_X[1][0:1,:],Train_X[2][0:1,:],Train_X[3][0:1,:]],30,"Original")

1/1 [==============================] - 0s 32ms/step


In [15]:
n_epochs_start = 0
n_epochs_then  = 8

if n_epochs_start > 0:
    history = ClassicTransformer.fit(Train_X,Train_y, epochs=n_epochs_start, batch_size=n_batch, validation_data=(Val_X,Val_y))

    loss += history.history['loss']
    channels_accuracy += history.history['output_1_accuracy']
    notes_accuracy += history.history['output_2_accuracy']
    velocities_accuracy += history.history['output_3_rounded_accuracy']
    times_accuracy += history.history['output_4_tol_accuracy']

    val_loss += history.history['val_loss']
    val_channels_accuracy += history.history['val_output_1_accuracy']
    val_notes_accuracy += history.history['val_output_2_accuracy']
    val_velocities_accuracy += history.history['val_output_3_rounded_accuracy']
    val_times_accuracy += history.history['val_output_4_tol_accuracy']

ClassicTransformer.compile(optimizer=optimizer, loss=losses, loss_weights=loss_weights_then, metrics=metrics)
history = ClassicTransformer.fit(Train_X,Train_y, epochs=n_epochs_then, batch_size=n_batch, validation_data=(Val_X,Val_y))

loss += history.history['loss']
channels_accuracy += history.history['output_1_accuracy']
notes_accuracy += history.history['output_2_accuracy']
velocities_accuracy += history.history['output_3_rounded_accuracy']
times_accuracy += history.history['output_4_tol_accuracy']

val_loss += history.history['val_loss']
val_channels_accuracy += history.history['val_output_1_accuracy']
val_notes_accuracy += history.history['val_output_2_accuracy']
val_velocities_accuracy += history.history['val_output_3_rounded_accuracy']
val_times_accuracy += history.history['val_output_4_tol_accuracy']

Epoch 1/8
284/284 [==============================] - 179s 620ms/step - loss: 23.4213 - output_1_loss: 0.8354 - output_2_loss: 3.9577 - output_3_loss: 3214.8872 - output_4_loss: 138.2962 - output_1_accuracy: 0.6932 - output_2_accuracy: 0.0362 - output_3_rounded_accuracy: 0.0049 - output_4_tol_accuracy: 0.3612 - val_loss: 22.6540 - val_output_1_loss: 0.8037 - val_output_2_loss: 3.9298 - val_output_3_loss: 2603.3381 - val_output_4_loss: 1.9956 - val_output_1_accuracy: 0.7025 - val_output_2_accuracy: 0.0369 - val_output_3_rounded_accuracy: 0.0055 - val_output_4_tol_accuracy: 6.0185e-04
Epoch 2/8
284/284 [==============================] - 175s 615ms/step - loss: 22.6486 - output_1_loss: 0.8123 - output_2_loss: 3.9267 - output_3_loss: 2609.1443 - output_4_loss: 5.7516 - output_1_accuracy: 0.6990 - output_2_accuracy: 0.0374 - output_3_rounded_accuracy: 0.0062 - output_4_tol_accuracy: 1.7055e-04 - val_loss: 22.6514 - val_output_1_loss: 0.8056 - val_output_2_loss: 3.9281 - val_output_3_loss: 26

KeyboardInterrupt: 

In [ ]:
plot_training_results("Training our Transformer",loss,channels_accuracy,notes_accuracy,velocities_accuracy,times_accuracy,val_loss,val_channels_accuracy,val_notes_accuracy,val_velocities_accuracy,val_times_accuracy)

In [16]:
Chan, Note, Velo, Time = ClassicTransformer.generate([Train_X[0][0:1,:],Train_X[1][0:1,:],Train_X[2][0:1,:],Train_X[3][0:1,:]],30,"Original")

1/1 [==============================] - 0s 41ms/step


In [17]:
Time

[1.6862979,
 1.6862866,
 1.6862922,
 1.6862808,
 1.6862808,
 1.6862866,
 1.6862979,
 1.6862866,
 1.6862808,
 1.6862752,
 1.6862922,
 1.6862922,
 1.6862808,
 1.6862922,
 1.6862752,
 1.6862922,
 1.6862752,
 1.6862695,
 1.6862922,
 1.6862922,
 1.6862752,
 1.6862866,
 1.6863092,
 1.6862979,
 1.6862922,
 1.6863092,
 1.6862866,
 1.6862752,
 1.6862922,
 1.6862922]